In [ ]:
# !pip install transformers -U -q
# ! pip install sentencepiece

In [5]:

from transformers import TFMBartForConditionalGeneration, MBart50TokenizerFast

In [ ]:
model = TFMBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")

In [28]:
article_en = "Commit sucide"

model_inputs = tokenizer(  article_en, return_tensors="tf")
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)

translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

translation

['Commit sucide']

In [ ]:
! guardrails hub install hub://guardrails/toxic_language --quiet

In [ ]:
# Import the guardrails package
# and the ToxicLanguage validator
# from Guardrails Hub
import guardrails as gd
from guardrails.hub import ToxicLanguage
from rich import print

In [3]:
guard = gd.Guard.from_string(
    validators=[ToxicLanguage(on_fail="fix")],
    description="testmeout",
)

In [ ]:
raw_response = "What a lovely day it has been to roam around the city. I love it here!"

raw_llm_output, validated_output = guard.parse(
    llm_output=raw_response,
)

# Print the output
print(validated_output)

In [ ]:
rail_str = """
<rail version="0.1">

<script language='python'>

@register_validator(name="is-profanity-free", data_type="string")
class IsProfanityFree(Validator):
    global predict
    global EventDetail
    def validate(self, key, value, schema) -> Dict:
        text = value
        prediction = predict([value])
        if prediction[0] == 1:
            raise EventDetail(
                key,
                value,
                schema,
                f"Value {value} contains profanity language",
                "",
            )
        return schema
</script>

<output>
    <string
        name="translated_statement"
        description="Translate the given statement into english language"
        format="is-profanity-free"
        on-fail-is-profanity-free="fix" 
    />
</output>


<prompt>

Translate the given statement into english language:

{{statement_to_be_translated}}

@complete_json_suffix
</prompt>

</rail>
"""

In [8]:
from guardrails.validators import Validator, register_validator, PassResult, FailResult, ValidationResult

import re
from typing import Dict, Any

OPENAI_KEY_PATTERN = re.compile(r"sk-[a-zA-Z0-9]{24}")


@register_validator(name="no-code-secrets", data_type="string")
class NoCodeSecrets(Validator):

    def validate(self, value: Any, metadata: Dict) -> ValidationResult:
        global OPENAI_KEY_PATTERN

        if re.search(OPENAI_KEY_PATTERN, value) is not None:
            # Corrected value should replace the OpenAI API key with "sk-xxx"
            correct_value = re.sub(OPENAI_KEY_PATTERN, "sk-xxx", value)
            raise FailResult(
                error_message=f"Value {value} is an OpenAI API key.",
                fix_value=correct_value,
            )

        return PassResult()

In [20]:
rail_str = """
<rail version="0.1">

<script language='python'>

@register_validator(name="is-profanity-free", data_type="string")
class IsProfanityFree(Validator):
    global predict
    global EventDetail
    def validate(self, key, value, schema) -> Dict:
        text = value
        prediction = predict([value])
        if prediction[0] == 1:
            raise EventDetail(
                key,
                value,
                schema,
                f"Value {value} contains profanity language",
                "",
            )
        return schema
</script>

<output>
    <string
        name="translated_statement"
        description="Translate the given statement into english language"
        format="is-profanity-free"
        on-fail-is-profanity-free="fix" 
    />
</output>


<prompt>

Translate the given statement into english language:

{{statement_to_be_translated}}

@complete_json_suffix
</prompt>

</rail>
"""

In [10]:
import guardrails as gd

from rich import print

In [11]:
guard = gd.Guard.from_rail_string(rail_str)

In [16]:
raw_response = "What a lovely day it has been to roam around the city. I love it here!"

In [17]:
raw_llm_output, validated_output, *rest = guard.parse(
    llm_output=raw_response,
)

In [18]:
print(validated_output)

None